# Intro to ETL with Python

Welcome! In this notebook we'll **generate fake, messy data** and walk step‑by‑step through a lightweight **ETL (Extract → Transform → Load)** process using **pandas**.

1. **Generate dirty data** – produce a CSV full of common data quality issues
2. **E**xtract – pull the raw CSV into pandas  
3. **T**ransform – clean, standardise & validate the data  
4. **L**oad – save the tidy result for downstream use
5. **Wrap it up** – automate everything in a simple, reusable pipeline function

> Feel free to run the notebook cell‑by‑cell or `Run All` to watch the pipeline in action.


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

pd.set_option('display.float_format', '{:.2f}'.format)

## 1 – Generate dirty data

In [3]:
# Reproducibility
random.seed(42)
np.random.seed(42)

N_ROWS = 120

# Helper to create messy dates
def random_date():
    base = datetime.today() - timedelta(days=random.randint(0, 365*3))
    fmt = random.choice(['%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y', '%d %b %Y', '%b %d, %Y'])
    return base.strftime(fmt)

cities = ['Riyadh', 'Jeddah', 'Dammam', 'Mecca', 'Medina']
products = ['Widget', 'Gizmo', 'Doodad', 'Thingamajig']

rows = []
for i in range(N_ROWS):
    rows.append({
        'order_id': i // 2 if random.random() < 0.15 else i,  # introduce duplicates
        'customer_name': random.choice(['Alice', 'Bob', 'Charlie', 'Dana', 'Eli']).lower().title(),
        'product': random.choice(products),
        'order_date': random_date(),
        'amount': random.choice([np.nan, -1, 0, 1]) * random.random() * 100 if random.random()<0.1 else random.random()*1000,
        'city': random.choice(cities + [c + '  ' for c in cities]).lower().title()
    })

dirty_df = pd.DataFrame(rows)

# Sprinkle extra duplicate rows
dirty_df = pd.concat([dirty_df, dirty_df.sample(5, random_state=1)], ignore_index=True)

dirty_df.to_csv('dirty_sales.csv', index=False)
dirty_df.head()

,order_id,customer_name,product,order_date,amount,city
0,0,Alice,Doodad,05/01/2024,102.50,Mecca
1,0,Dana,Widget,2025-03-20,505.36,Riyadh
2,2,Eli,Thingamajig,24 Feb 2024,809.43,Riyadh
3,3,Bob,Thingamajig,06-24-2023,957.21,Riyadh
4,2,Dana,Widget,05-16-2023,807.13,Dammam


## 2 – Extract

In [4]:
raw_df = pd.read_csv('dirty_sales.csv')
print(f"Rows: {len(raw_df)} (including duplicates)")
raw_df.head()

Rows: 125 (including duplicates)


,order_id,customer_name,product,order_date,amount,city
0,0,Alice,Doodad,05/01/2024,102.50,Mecca
1,0,Dana,Widget,2025-03-20,505.36,Riyadh
2,2,Eli,Thingamajig,24 Feb 2024,809.43,Riyadh
3,3,Bob,Thingamajig,06-24-2023,957.21,Riyadh
4,2,Dana,Widget,05-16-2023,807.13,Dammam


## 3 – Transform

In [5]:
def clean_sales(df: pd.DataFrame) -> pd.DataFrame:
    """Clean raw sales DataFrame and return a tidy version."""
    out = df.copy()

    # --- Remove duplicate order_id rows (keep first) ---
    out = out.drop_duplicates(subset=['order_id'])

    # --- Strip whitespace and standardise case on city ---
    out['city'] = out['city'].str.strip().str.title()

    # --- Standardise dates to ISO format ---
    out['order_date'] = pd.to_datetime(out['order_date'], errors='coerce')

    # --- Fix amounts ---
    amt_median = out['amount'].median(skipna=True)
    out['amount'] = out['amount'].abs()  # make negative values positive
    out['amount'] = out['amount'].fillna(amt_median)

    # --- Final sanity checks ---
    out = out.dropna(subset=['order_id', 'order_date'])

    return out

clean_df = clean_sales(raw_df)
clean_df.head()

,order_id,customer_name,product,order_date,amount,city
0,0,Alice,Doodad,2024-05-01,102.50,Mecca
26,26,Dana,Gizmo,2023-07-01,249.81,Jeddah
63,63,Charlie,Widget,2024-10-08,693.95,Mecca
67,67,Dana,Gizmo,2022-06-10,534.12,Jeddah


## 4 – Load

In [6]:
clean_df.to_csv('clean_sales.csv', index=False)
print("Clean file written: clean_sales.csv")
print(f"Clean rows: {len(clean_df)}")

Clean file written: clean_sales.csv
Clean rows: 4


## 5 – Automate with a pipeline function

In [7]:
def run_etl(input_path: str = 'dirty_sales.csv',
             output_path: str = 'clean_sales.csv',
             verbose: bool = True):
    """Simple, reproducible ETL pipeline."""
    if verbose: print('Extract ▶ ', end='')
    df_raw = pd.read_csv(input_path)
    if verbose: print('Transform ▶ ', end='')
    df_clean = clean_sales(df_raw)
    if verbose: print('Load ▶ ', end='')
    df_clean.to_csv(output_path, index=False)
    if verbose: print('Done!')
    return df_clean

# Run the complete pipeline
_ = run_etl()

Extract ▶ Transform ▶ Load ▶ Done!


### 🎉 All done!

You now have two CSV files:

* **dirty_sales.csv** – intentionally messy data
* **clean_sales.csv** – fully cleaned, ready for analytics

Feel free to tweak the `clean_sales` function or extend the pipeline (e.g. load to a database, schedule via Airflow, etc.) to better suit your goals.

---


# Another Example

In [3]:

# a simple ETL (Extract, Transform, Load) process
# using pandas with a publicly available dataset from Kaggle.

# We'll use a popular dataset from Kaggle: "Titanic: Machine Learning from Disaster"
# It's good for beginners as it has missing values, different data types, and inconsistencies.

# You can download the train.csv file directly from the Kaggle dataset page
# (https://www.kaggle.com/c/titanic/data) or use the command line to download it.

# For this example, we'll assume the file 'train.csv' is already in your Colab environment.
# If not, you might need to upload it or download it programmatically.

# --- 1. Extract ---
# The Extract step is about getting the data from its source.
# In this case, our source is a CSV file.
print("--- Starting ETL Process ---")
print("1. Extracting data...")
try:
    # Read the CSV file into a pandas DataFrame
    df_raw = pd.read_csv('train.csv')
    print(f"Successfully loaded {len(df_raw)} rows.")
    print("Raw data preview:")
    print(df_raw.head())
    print("\nRaw data info:")
    df_raw.info()
    print("-" * 30)

except FileNotFoundError:
    print("Error: 'train.csv' not found. Please upload the file or check the path.")
    # Exit the process if the file is not found
    exit()


# --- 2. Transform ---
# The Transform step involves cleaning, standardizing, and validating the data.
# Let's perform some common cleaning tasks on the Titanic dataset.

print("2. Transforming data...")

# Create a copy to avoid modifying the raw data directly
df_clean = df_raw.copy()

# Task 1: Handle Missing Values
# The 'Age', 'Embarked', and 'Cabin' columns have missing values.
# Let's fill 'Age' with the median and 'Embarked' with the most frequent value.
# We'll drop the 'Cabin' column as it has too many missing values and might not be useful for simple analysis.

print("  Handling missing values...")
# Fill missing 'Age' with the median age
median_age = df_clean['Age'].median()
df_clean['Age'].fillna(median_age, inplace=True)
print(f"    Filled missing 'Age' with median: {median_age:.2f}")

# Fill missing 'Embarked' with the most frequent value (mode)
most_frequent_embarked = df_clean['Embarked'].mode()[0]
df_clean['Embarked'].fillna(most_frequent_embarked, inplace=True)
print(f"    Filled missing 'Embarked' with mode: {most_frequent_embarked}")

# Drop the 'Cabin' column
df_clean.drop('Cabin', axis=1, inplace=True)
print("    Dropped 'Cabin' column due to many missing values.")

# Check if missing values are handled
print("  Checking remaining missing values:")
print(df_clean.isnull().sum())
print("-" * 30)


# Task 2: Standardize Data
# Convert 'Sex' to numerical representation (0 for female, 1 for male)
print("  Standardizing data...")
df_clean['Sex'] = df_clean['Sex'].map({'female': 0, 'male': 1})
print("    Converted 'Sex' to numerical (0: female, 1: male).")
print("-" * 30)

# Task 3: Create New Features (Feature Engineering - optional but common in ETL)
# Create a new feature 'FamilySize' from 'SibSp' and 'Parch'
df_clean['FamilySize'] = df_clean['SibSp'] + df_clean['Parch'] + 1 # +1 for the passenger themselves
print("  Creating new features: 'FamilySize'.")
print("-" * 30)


# Task 4: Data Type Conversion (often done implicitly during loading, but good to be aware)
# pandas usually infers data types correctly, but you can explicitly convert if needed.
# For instance, ensuring 'PassengerId' is an integer.
# df_clean['PassengerId'] = df_clean['PassengerId'].astype(int)
print("  Ensuring correct data types (pandas usually handles this).")
print("-" * 30)


print("Transformation complete. Cleaned data preview:")
print(df_clean.head())
print("\nCleaned data info:")
df_clean.info()
print("-" * 30)


# --- 3. Load ---
# The Load step is about storing the transformed data in its final destination.
# We'll save the cleaned data to a new CSV file.

print("3. Loading data...")
output_filename = 'cleaned_titanic_data.csv'
df_clean.to_csv(output_filename, index=False)
print(f"Successfully saved cleaned data to '{output_filename}'.")
print("-" * 30)

print("--- ETL Process Finished ---")

# You can now inspect the 'cleaned_titanic_data.csv' file in your Colab environment.
# The data is now cleaner and ready for analysis or modeling!



--- Starting ETL Process ---
1. Extracting data...
Successfully loaded 891 rows.
Raw data preview:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male 22.00      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female 38.00      1   
2                             Heikkinen, Miss. Laina  female 26.00      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female 35.00      1   
4                           Allen, Mr. William Henry    male 35.00      0   

   Parch            Ticket  Fare Cabin Embarked  
0      0         A/5 21171  7.25   NaN        S  
1      0          PC 17599 71.28   C85        C  
2      0  STON/O2. 3101282  7.92   NaN        S  
3      0            1

<ipython-input-3-1649bce99ed4>:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Age'].fillna(median_age, inplace=True)
<ipython-input-3-1649bce99ed4>:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin